## 理論1層: 熱による進化

- 物語: 鍛冶屋は鉄を高温で熱しては、金槌ちで叩き、水で冷やし、繰り返し、良い鉄に仕上げる
- 動画：[外部リンク：福島県森林環境文化記録映像【第８章　野鍛冶】](https://www.youtube.com/watch?v=IRPxZa1NZbM)
- なぜ：最適化（進化に似てる）が成長や教育のような最重要の概念であり、説明が最もコスパがある
- 特徴: 大域進化、機械評価、自動探索、汎用
- 注意: 多くの時間を消費する

## シミュレーション (焼きなまし法)

### 計算手順

- 計算できる正しさがある(評価関数)
- 高い温度がある
- 無作為の「状態」から始まる
- a. 今の状態の近くに次の状態がある。近くから選ぶ(近傍)
- 1. 次の状態が良い状態なら変化する
- 2. 高い温度なら高確率で変化する。低い温度なら低確率で変化する
- 温度は下がる
- a.から繰り返す
- 最終的に評価の高い変化しにくい状態となる

### 巡回セールスマン(販売員)問題で焼き鈍し法を体験

- 物語：販売員が営業先の拠点を全て回るが、楽をしたい
- 動画：[外部リンク：巡回セールマン問題解説](https://www.youtube.com/watch?v=GEo8wqZroSE)
- 理論的モデル：各点を最小の距離になるように結ぶ
- 評価関数：営業先の拠点の訪問順による総距離
- 近く（近傍）：営業先の拠点の訪問順を交換する
    - 3番目の拠点、2番目、 1番目に訪れるなら[3,2,1]で、 1,2で交換 [2,3,1] 1,3で交換 [1,2,3], 2,3 で交換 [3,1,2]となります
	- [1, 3, 2] [2, 1 ,3] は　近くありません

### 手順

- 上の理論から想像しましょう。どうなるでしょうか
    - 下にプログラムがありますので詳細を確認してもよいかもしれません
- VsCode: Run All(実行)
- web版: 上部メニュー ＞ Kernel(核：中心部：カーネル) ＞ Restart ＆ Run All(再スタート？実行) を押す
- 画像を比較する。総距離を比較する。考察する
- 興味があれば、媒介変数(例:NUM=12)・プログラムを改変して、手順を再度行います

### プログラミング初心者向け

- 下記プログラムを写経する
    - コピペじゃなくて、手書きで写す
    - このアプリ、または、vscodeでやってみよう
- 実行して、間違いを探して、繰り返してみよう


In [ ]:
# 言語 python3

import random
import numpy as np
import math
from PIL import Image, ImageDraw

WIDTH = 300  # 図の幅
HEIGHT = 300 # 図の高さ
REPORT_TIMES = 10 # 情報を出力する回数
REPORT_ITER = 10000 # 情報を出力するループ回数
MAX_ITER = REPORT_TIMES * REPORT_ITER # 最大ループ

###################################################
# 巡回セールスマン(販売員)問題
class TSP:
    # 定数
    NUM = 12 # 拠点の数

    # 初期化
    def __init__(self):
        order = list(range(self.NUM)) # 巡る順番(状態) [0..NUM-1]
        random.shuffle(order)
        self.order = np.asarray(order)
        self.distCache = makeArray2D(self.NUM, self.NUM, 0) # 点から点への距離 NUM*NUMの配列 Array[[距離,...],...]
        vertexPos = []
        for i in range(self.NUM):
            vertexPos.append([0, 0])
            vertexPos[i][0] = random.random()
            vertexPos[i][1] = random.random()
        self.vertexPos = np.asarray(vertexPos, dtype=np.float64) # 拠点の位置: [(x,y)...]
        self.calcDistanceCache()

    # 各点の距離
    def calcDistanceCache(self):
        for i in range(self.order.size):
            for j in range(self.order.size):
                if(i == j):
                    self.distCache[i][j] = 0
                else:
                    self.distCache[i][j] = squareRoot(self.vertexPos[i][0], self.vertexPos[j][0], self.vertexPos[i][1], self.vertexPos[j][1])

    # 総距離
    def calcTotalDistance(self, order):
        total = 0
        for i in range(order.size - 1):
            total += self.distCache[order[i]][order[i+1]]
        total += self.distCache[order[order.size - 1]][order[0]]
        return total
    
    # interface
    def calcScore(self, state):
        return self.calcTotalDistance(state)
    
    def getState(self):
        return self.order
    
    def setState(self, state):
        self.order = state
    
    def draw(self):
        im = Image.new('RGB', (WIDTH, HEIGHT), (192, 192, 192))
        drawObj = ImageDraw.Draw(im)
        for i in range(1, self.order.size):
            drawLine(drawObj, self.vertexPos[self.order[i-1]][0], self.vertexPos[self.order[i-1]][1], self.vertexPos[self.order[i]][0], self.vertexPos[self.order[i]][1])
        drawLine(drawObj, self.vertexPos[self.order[self.order.size-1]][0], self.vertexPos[self.order[self.order.size-1]][1], self.vertexPos[self.order[0]][0], self.vertexPos[self.order[0]][1])
        im.show()

###################################################
# 焼き鈍し法
class SA:
    TEMP_DECAY_FACTOR = 0.15 # 温度の設定値
    # 初期化
    def __init__(self, problem):
        self.iteration = 0  # ループした回数
        self.temp = 0       # 温度
        self.score = 0
        self.problem = problem
        self.minState = self.problem.getState().copy()
        self.minScore = 100000
        self.n = self.problem.getState().size

    # 全体の流れ
    def solve(self):
        self.temp = self.temperature()
        self.calcNext()
        self.score = self.problem.calcScore(self.problem.getState())
        if (self.score < self.minScore):
            self.minScore = self.score
            self.minState = self.problem.getState()
        self.iteration += 1

    #　温度
    def temperature(self):
        return self.TEMP_DECAY_FACTOR ** (self.iteration / MAX_ITER)

    # 近傍から次の状態へ遷移する
    def calcNext(self):
        for i in range(0, self.n - 1):
            for j in range(i + 1, self.n - 1):
                scoreBefore = self.problem.calcScore(self.problem.getState())
                nextState = swap(self.problem.getState().copy(), i, j)
                scoreAfter = self.problem.calcScore(nextState)
                subScore = scoreAfter - scoreBefore
                if(subScore < 0):
                    self.problem.setState(nextState)
                else:
                    prob = self.calcProbability(subScore)
                    if (random.random() <= prob):
                        self.problem.setState(nextState)

    # 温度による遷移確率
    def calcProbability(self, subScore):
        return math.exp((-1 * subScore)  / self.temp)

###################################################
# 出力
def report(solver):
    print("ーーーーーーーーーーーーー")
    print("繰返回数 : "+ str(solver.iteration))
    print("温度 : "+ str(solver.temp))
    print("総距離 : "+  str(solver.score))
    print("最小距離 : "+ str(solver.minScore))
    solver.problem.draw()

# 出力
def reportEnd(solver):
    print("ーーーーーーーーーーーーー")
    print("最小距離 : "+ str(solver.minScore))
    solver.problem.setState(solver.minState)
    solver.problem.draw()

###################################################

def drawLine(drawObj, beforeX, beforeY, afterX, afterY):
    drawObj.line([(int(beforeX* WIDTH), int(beforeY*HEIGHT)), (int(afterX*WIDTH), int(afterY*HEIGHT))], fill="blue", width=1)

# 二次元配列作成
def makeArray2D(n, m, value):
    ret = []
    for i in range(n):
        ret.append(list(range(m)))
        for j in range(m):
            ret[i][j] = value
    return np.asarray(ret, dtype=np.float64) # 高速化のため配列

# 入れ替え
def swap(arr, i, j):
    tmp = arr[i]
    arr[i] = arr[j]
    arr[j] = tmp
    return arr

def twice(x):
    return x * x

def squareRoot(ax, bx, ay, by):
    return math.sqrt(twice(ax - bx) + twice(ay - by))

###################################################
random.seed() # 乱数初期化
solver = SA(TSP())     # 初期化
solver.problem.draw()
for i in range(REPORT_TIMES):
    for j in range(REPORT_ITER):
        solver.solve()
    report(solver)
reportEnd(solver)

### 考察（要検証）


### 閃き方法 (要検証)

- 熱力学
- 鍛冶屋

### 生活の知恵　（要検証）



### 参考文献

- [焼きなまし法(Simulated Annealing)と2-Opt法による巡回セールスマン問題の解法](https://salad-bowl-of-knowledge.github.io/hp/processing/2017/12/03/simulated_annealing.html)


### 次

- [理論2層：近傍](Theory2_1.ipynb)
- [理論2層：合成](Theory2_2.ipynb)
- [理論2層：汎用](Theory2_3.ipynb)
- [理論2と人：評価](Theory2_4.ipynb)
- [理論2と創造](Theory2_5.ipynb)
